### **一、加载常用库**

In [1]:
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

### **二、处理交易数据，生成stktrd**

In [2]:
#计算半衰期指数权重,时间序列长度为windows，半衰期为half
def Weihgt_half(windows,half):
    list_W = []
    for i in range(windows):
        locals()['W'+str(i)] = 0.5**((windows+1-i)/half)
        list_W.append(locals()['W'+str(i)])
        
    return list_W/pd.Series(list_W).sum()#返回一个数组

#计算半衰加权的函数,column为被加权的列，windows,half为权重计算参数，new_col为加权结果列
def Mean_half(df,column,new_col,windows,half):
    result = pd.DataFrame()
    codelist = df.code.unique()
    Weihgt_h = Weihgt_half(windows, half)
    for code in codelist:
        TRD_sub = df[df.code == code].copy()
        TRD_sub.loc[:,[new_col]] = TRD_sub.rolling(windows)[column].apply(lambda x: (x*Weihgt_h).sum())
        result = pd.concat([result,TRD_sub])
    return result

In [3]:
#读取股票日交易数据
stktrd = pd.read_csv("D:/191alphas/stkdata/alldata.csv", encoding="gbk", usecols=['Stkcd','Trddt','Clsprc','Dnshrtrd','Dsmvosd','Dsmvtll','Dretwd'])

In [4]:
#修改文件时间格式
stktrd['Trddt'] = pd.to_datetime(stktrd['Trddt']) #时间格式

In [5]:
#修改列名
stktrd.columns = ['code','date','close','volume','value_neg','value_all','return_day']

In [6]:
#计算流通股本shares_neg
stktrd.loc[:,['shares_neg']] = stktrd['value_neg']/stktrd['close']

In [7]:
stktrd.loc[:,'year'] = stktrd['date'].dt.year
stktrd.loc[:,'month'] = stktrd['date'].dt.month

In [8]:
stktrd

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,year,month
0,1,2010-01-04,23.710,24192276,69330749.18,73629834.50,-0.027082,2.924114e+06,2010,1
1,1,2010-01-05,23.300,55649982,68131862.33,72356606.65,-0.017292,2.924114e+06,2010,1
2,1,2010-01-06,22.900,41214313,66962216.62,71114433.15,-0.017167,2.924114e+06,2010,1
3,1,2010-01-07,22.650,35533685,66231188.06,70338074.71,-0.010917,2.924114e+06,2010,1
4,1,2010-01-08,22.600,28854306,66084982.34,70182803.02,-0.002208,2.924114e+06,2010,1
...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.621,479100,114264.00,216729.00,-0.026646,1.840000e+05,2022,1
8672605,900957,2022-01-25,0.611,372779,112424.00,213239.00,-0.016103,1.840000e+05,2022,1
8672606,900957,2022-01-26,0.614,222600,112976.00,214286.00,0.004910,1.840000e+05,2022,1
8672607,900957,2022-01-27,0.612,245100,112608.00,213588.00,-0.003257,1.840000e+05,2022,1


In [9]:
#读取月度收益数据
return_m = pd.read_csv('D:/191alphas/return_m/return_m.csv')

In [10]:
#时间格式
return_m.loc[:,'Trdmnt'] = pd.to_datetime(return_m.loc[:,'Trdmnt'])

In [11]:
#提取年月
return_m.loc[:,'year'] = return_m.Trdmnt.dt.year
return_m.loc[:,'month'] = return_m.Trdmnt.dt.month

In [12]:
#重命名列
return_m.columns = ['code','Trdmnt','return_m','year','month']

In [13]:
#计算对数收益率ln（1+return_m）
return_m.loc[:,'return_m_ln'] = np.log(return_m.return_m+1)

In [14]:
#计算过去12个月的对数月度收益的累计值Z_12
return_m.loc[:,'Z_12'] = return_m.groupby('code',as_index = False)['return_m_ln'].rolling(12).sum()['return_m_ln']

#### **计算Volatility-CMRA指标**

In [15]:
#由于Z_12存在较多小于-1的值，故用Z_12代替ln(1+Z_12)值进行计算CMRA
return_m.loc[:,'CMRA'] = return_m.groupby(['code'],as_index = False)['Z_12'].rolling(12,min_periods=2).max()['Z_12'] - return_m.groupby(['code'],as_index = False)['Z_12'].rolling(12,min_periods=2).min()['Z_12']

In [16]:
#去除return_m不需要的列
return_m.drop(columns = ['Trdmnt','return_m','return_m_ln','Z_12'], inplace = True)

In [17]:
#合并月度收益
stktrd = pd.merge(stktrd,return_m, on = ['code','year','month'], how = 'left')

In [18]:
#删除stktrd不需要的列
stktrd.drop(columns=['year','month'], inplace=True)

In [19]:
#由于计算CMRA需要24个月的月度收益率，设置min_periods=2后需要14个月的月度收益，所以上市不足14个月的股票CMRA当前值为空，未来将被剔除
stktrd.isnull().sum()

code                0
date                0
close               0
volume              0
value_neg           0
value_all           0
return_day          0
shares_neg          0
CMRA          1002684
dtype: int64

In [20]:
#由于计算CMRA需要24个月的月度收益率，设置min_periods=2后需要14个月的月度收益，所以上市不足14个月的股票CMRA当前值为空，未来将被剔除
return_m_dropna = return_m.dropna()
len(return_m_dropna.code.unique())

4269

In [21]:
stktrd

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,CMRA
0,1,2010-01-04,23.710,24192276,69330749.18,73629834.50,-0.027082,2.924114e+06,1.603651
1,1,2010-01-05,23.300,55649982,68131862.33,72356606.65,-0.017292,2.924114e+06,1.603651
2,1,2010-01-06,22.900,41214313,66962216.62,71114433.15,-0.017167,2.924114e+06,1.603651
3,1,2010-01-07,22.650,35533685,66231188.06,70338074.71,-0.010917,2.924114e+06,1.603651
4,1,2010-01-08,22.600,28854306,66084982.34,70182803.02,-0.002208,2.924114e+06,1.603651
...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.621,479100,114264.00,216729.00,-0.026646,1.840000e+05,NaN
8672605,900957,2022-01-25,0.611,372779,112424.00,213239.00,-0.016103,1.840000e+05,NaN
8672606,900957,2022-01-26,0.614,222600,112976.00,214286.00,0.004910,1.840000e+05,NaN
8672607,900957,2022-01-27,0.612,245100,112608.00,213588.00,-0.003257,1.840000e+05,NaN


### **三、处理财报数据，生成finreport**

#### **1、处理利润表和资产负债表数据**

In [22]:
#读取资产负债表和利润表需要的数据
balance = pd.read_csv("D:/191alphas/balance/FS_Combas.csv", usecols=['Stkcd','Accper','Typrep','A001000000','A002000000','A002206000','A003100000'])
profit = pd.read_csv("D:/191alphas/profit/FS_Comins.csv", usecols=['Stkcd','Accper','Typrep','B001100000','B001200000','B002100000','B002000101'])

In [23]:
#时间格式转换
balance['Accper'] = pd.to_datetime(balance['Accper'])
profit['Accper'] = pd.to_datetime(profit['Accper'])

In [24]:
#删除母公司报表数据B，保留合并报表数据A
balance.drop(balance[balance['Typrep']=='B'].index, inplace=True)
profit.drop(profit[profit['Typrep']=='B'].index, inplace=True)

In [25]:
#删除Typrep列
balance.drop(columns='Typrep', inplace=True)
profit.drop(columns='Typrep', inplace=True)

In [26]:
#修改列名
balance.columns=['code','Accper','asset','debt_long','debt_all','equity']
profit.columns=['code','Accper','sales','costs','tax','netincome']

In [27]:
#利润表提取年、季度、月
profit.loc[:,'year'] = profit['Accper'].dt.year
profit.loc[:,'quarter'] = profit['Accper'].dt.quarter
profit.loc[:,'month'] = profit['Accper'].dt.month

In [28]:
#删除利润表month为1的数据
profit.drop(profit[profit['month']==1].index, inplace=True)

In [29]:
#差分计算每季度利润表的数据
profit.loc[:,['sales_dif']] = profit.groupby(['code','year'])['sales'].diff()
profit.loc[:,['costs_dif']] = profit.groupby(['code','year'])['costs'].diff()
profit.loc[:,['tax_dif']] = profit.groupby(['code','year'])['tax'].diff()
profit.loc[:,['netincome_dif']] = profit.groupby(['code','year'])['netincome'].diff()

In [30]:
#填充差分后的空值
profit['sales_dif'].fillna(profit['sales'], inplace = True)
profit['costs_dif'].fillna(profit['costs'], inplace = True)
profit['tax_dif'].fillna(profit['tax'], inplace = True)
profit['netincome_dif'].fillna(profit['netincome'], inplace = True)

In [31]:
#删除fstates不需要的列，并重命名sales_dif、costs_dif、tax_dif、profit_dif
profit.drop(columns = ['sales','costs','tax','netincome'] , inplace=True)

In [32]:
#重命名sales_dif、costs_dif、tax_dif、netincome_dif
profit.rename(columns={'sales_dif':'sales','costs_dif':'costs','tax_dif':'tax','netincome_dif':'netincome'}, inplace=True)

In [33]:
#合并profit和balance
finreport = pd.merge(profit,balance, on = ['code','Accper'], how = 'left')

In [34]:
#删除2010年以前的数据
finreport.drop(finreport[finreport['year'] < 2010].index, inplace = True)

In [35]:
#查看列空值个数，可以删除asset为空的行
finreport.isnull().sum()

code             0
Accper           0
year             0
quarter          0
month            0
sales           18
costs            0
tax           1348
netincome        0
asset           35
debt_long    22970
debt_all        35
equity          35
dtype: int64

In [36]:
#删除asset为nan的数据
finreport.dropna(subset=['asset'], inplace = True)

In [37]:
#重置索引
finreport.reset_index(drop = True, inplace = True)

In [38]:
#0填充空值，空值存在于sales、tax、debt_long
finreport.fillna(0, inplace = True)

In [39]:
finreport.isnull().sum()

code         0
Accper       0
year         0
quarter      0
month        0
sales        0
costs        0
tax          0
netincome    0
asset        0
debt_long    0
debt_all     0
equity       0
dtype: int64

In [40]:
#有部分新上市股票尚未发布财务报表，故不含有财报数据
len(finreport.code.unique())

4610

In [41]:
finreport

,code,Accper,year,quarter,month,sales,costs,tax,netincome,asset,debt_long,debt_all,equity
0,1,2010-03-31,2010,1,3,3.956690e+09,2.089101e+09,4.189030e+08,1.578118e+09,6.199276e+11,9.466200e+09,5.978178e+11,2.210983e+10
1,1,2010-06-30,2010,2,6,4.275850e+09,2.602182e+09,4.197250e+08,1.455001e+09,6.243982e+11,9.466373e+09,5.939771e+11,3.042111e+10
2,1,2010-09-30,2010,3,9,4.481900e+09,2.548856e+09,4.412770e+08,1.700821e+09,6.750639e+11,9.467785e+09,6.429197e+11,3.214414e+10
3,1,2010-12-31,2010,4,12,4.847392e+09,2.932584e+09,4.343000e+08,1.549876e+09,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10
4,1,2011-03-31,2011,1,3,5.617613e+09,2.755081e+09,6.669000e+08,2.402199e+09,8.075634e+11,9.474041e+09,7.716134e+11,3.595007e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142287,688981,2021-12-31,2021,4,12,1.025964e+10,8.515152e+09,3.382580e+08,3.414822e+09,2.299328e+11,3.594267e+10,6.795798e+10,1.091988e+11
142288,689009,2020-12-31,2020,4,12,6.002741e+09,5.835464e+09,7.872374e+07,7.347313e+07,6.557638e+09,2.480000e+07,2.860859e+09,3.696778e+09
142289,689009,2021-03-31,2021,1,3,1.778577e+09,1.761040e+09,-3.185251e+06,2.540758e+07,6.618843e+09,2.480000e+07,2.873484e+09,3.745150e+09
142290,689009,2021-06-30,2021,2,6,2.940142e+09,2.630378e+09,9.346863e+07,2.285331e+08,7.581717e+09,4.484897e+07,3.560179e+09,4.021849e+09


#### **2、netincome_ttm4计算，for EarningsYield-ETOP**

In [42]:
#计算最近12个月的公司净利润，即netincome最近四个季度的ttm
finreport.loc[:,'netincome_ttm4'] = finreport.groupby(['code'],as_index=False)['netincome'].rolling(4).sum()['netincome']

In [43]:
#计算netincome_ttm4需要最近四季度财报，故财报发布不足四期的股票netincome_ttm4值为nan
finreport.isnull().sum()

code                  0
Accper                0
year                  0
quarter               0
month                 0
sales                 0
costs                 0
tax                   0
netincome             0
asset                 0
debt_long             0
debt_all              0
equity                0
netincome_ttm4    13520
dtype: int64

#### **3、OCF_ttm4计算，for EarningsYield-CETOP**

In [44]:
#OCF（sales-costs-tax）计算
finreport.loc[:,'OCF'] = finreport['sales'] - finreport['costs'] - finreport['tax']

In [45]:
#计算最近12个月的公司净利润，即netincome最近四个季度的ttm
finreport.loc[:,'OCF_ttm4'] = finreport.groupby(['code'],as_index=False)['OCF'].rolling(4).sum()['OCF']

In [46]:
#删除不需要的列
finreport.drop(columns = ['sales','costs','tax','netincome','OCF'], inplace = True)

In [47]:
finreport

,code,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4
0,1,2010-03-31,2010,1,3,6.199276e+11,9.466200e+09,5.978178e+11,2.210983e+10,NaN,NaN
1,1,2010-06-30,2010,2,6,6.243982e+11,9.466373e+09,5.939771e+11,3.042111e+10,NaN,NaN
2,1,2010-09-30,2010,3,9,6.750639e+11,9.467785e+09,6.429197e+11,3.214414e+10,NaN,NaN
3,1,2010-12-31,2010,4,12,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,6.283816e+09,5.674904e+09
4,1,2011-03-31,2011,1,3,8.075634e+11,9.474041e+09,7.716134e+11,3.595007e+10,7.107897e+09,6.421850e+09
...,...,...,...,...,...,...,...,...,...,...,...
142287,688981,2021-12-31,2021,4,12,2.299328e+11,3.594267e+10,6.795798e+10,1.091988e+11,1.073310e+10,4.714949e+09
142288,689009,2020-12-31,2020,4,12,6.557638e+09,2.480000e+07,2.860859e+09,3.696778e+09,NaN,NaN
142289,689009,2021-03-31,2021,1,3,6.618843e+09,2.480000e+07,2.873484e+09,3.745150e+09,NaN,NaN
142290,689009,2021-06-30,2021,2,6,7.581717e+09,4.484897e+07,3.560179e+09,4.021849e+09,NaN,NaN


In [48]:
#删除finreport每只股前三行，计算netincome_ttm4	、OCF_ttm4需要最近四季度财报，故财报发布不足四期的股票netincome_ttm4、OCF_ttm4值为nan
finreport = finreport.groupby('code',as_index = False).apply(lambda x: x.drop(x.head(3).index))

In [49]:
finreport.isnull().sum()

code              0
Accper            0
year              0
quarter           0
month             0
asset             0
debt_long         0
debt_all          0
equity            0
netincome_ttm4    0
OCF_ttm4          0
dtype: int64

In [50]:
len(finreport.code.unique())

4319

#### **4、处理growth数据,SGRO和EGRO计算**

In [51]:
#导入增长数据，营业总收入增长率（06开始）F081701B、净利润增长率（06开始）F081003B
growth = pd.read_csv("D:/191alphas/growth/growth.csv", usecols=['Stkcd','Accper','Typrep','F081701B','F081003B'])

In [52]:
#删除母公司报表数据B，保留合并报表数据A
growth.drop(growth[growth['Typrep']=='B'].index, inplace=True)

In [53]:
#时间格式转换
growth['Accper'] = pd.to_datetime(growth['Accper'])

In [54]:
#提取年、季度
growth.loc[:,'year'] = growth['Accper'].dt.year
growth.loc[:,'quarter'] = growth['Accper'].dt.quarter

In [55]:
#去除不需要一二三季度的行，只保留年报增长数据
growth.drop(growth[growth['quarter'] != 4].index, inplace = True)

In [56]:
#列重命名
growth.columns = ['code','Accper','Typrep','sales_growth','netincome_growth','year','quarter']

In [57]:
#求5年复合增长率，考虑到上市不足5年的数据，用算术平均代替几何平均，不足5年的公司用最大上市年限的增速求平均
growth[['SGRO','EGRO']] = growth.groupby('code',as_index=False)[['sales_growth','netincome_growth']].rolling(5,min_periods=1).mean()[['sales_growth','netincome_growth']]

In [58]:
#删除2010年以前的数据
growth.drop(growth[growth['year'] < 2010].index,inplace = True)

In [59]:
#去除不需要的列
growth.drop(columns = ['Typrep','Accper','quarter','sales_growth','netincome_growth'], inplace = True)

In [60]:
#growth指标数据较全
growth_dropna = growth.dropna()
len(growth_dropna.code.unique())

4794

In [61]:
#重置索引
growth.reset_index(drop = True,inplace = True)

In [62]:
growth

,code,year,SGRO,EGRO
0,1,2010,0.321171,2.082371
1,1,2011,0.354519,1.672216
2,1,2012,0.318884,1.525509
3,1,2013,0.318949,1.704629
4,1,2014,0.383596,0.326071
...,...,...,...,...
41908,688981,2021,0.144340,1.469035
41909,689009,2017,NaN,NaN
41910,689009,2018,2.075106,-1.876590
41911,689009,2019,1.077368,-0.564364


In [63]:
##growth指标数据较全，多为上市不足一年的股票，即未发布年报的股票
growth.isnull().sum()

code       0
year       0
SGRO    2330
EGRO    2319
dtype: int64

#### **5、合并财报和增长数据，以及财报公布日数据**

In [64]:
#读取财报披露日数据
disclosure = pd.read_csv('D:/191alphas/disclosure/disclosure.csv')

In [65]:
#删除Stknme列
disclosure.drop(columns = 'Stknme',inplace = True)

In [66]:
#重命名
disclosure.columns = ['code','Accper','disclosure']

In [67]:
#时间类型转换
disclosure['Accper'] = pd.to_datetime(disclosure['Accper'])
disclosure['disclosure'] = pd.to_datetime(disclosure['disclosure'])

In [68]:
disclosure

,code,Accper,disclosure
0,1,1990-12-31,1991-04-30
1,1,1991-12-31,1992-04-30
2,1,1992-06-30,1992-07-28
3,1,1992-12-31,1993-02-26
4,1,1993-06-30,1993-08-05
...,...,...,...
206962,689009,2021-03-31,2021-04-30
206963,689009,2021-06-30,2021-08-25
206964,689009,2021-09-30,2021-10-26
206965,689009,2021-12-31,NaT


In [69]:
#disclosure为nan多为尚未发布财报的股票
disclosure.isnull().sum()

code             0
Accper           0
disclosure    8124
dtype: int64

In [70]:
#合并finreport与growth，on = ['code','growth'],选择取交集，则剔除了未发布过年报的数据
style_factor = pd.merge(finreport, growth, on = ['code','year'])

In [71]:
style_factor

,code,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4,SGRO,EGRO
0,1,2010-12-31,2010,4,12,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,6.283816e+09,5.674904e+09,0.321171,2.082371
1,1,2011-03-31,2011,1,3,8.075634e+11,9.474041e+09,7.716134e+11,3.595007e+10,7.107897e+09,6.421850e+09,0.354519,1.672216
2,1,2011-06-30,2011,2,6,8.520570e+11,1.312510e+10,8.139046e+11,3.815240e+10,7.982443e+09,7.191803e+09,0.354519,1.672216
3,1,2011-09-30,2011,3,9,1.207212e+12,1.604695e+10,1.135526e+12,6.970795e+10,9.237058e+09,8.623429e+09,0.354519,1.672216
4,1,2011-12-31,2011,4,12,1.258177e+12,1.605449e+10,1.182796e+12,7.331084e+10,1.027863e+10,9.709968e+09,0.354519,1.672216
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119712,688819,2021-12-31,2021,4,12,2.667708e+10,3.958674e+08,1.393798e+10,1.227199e+10,1.369366e+09,9.735234e+08,0.098626,0.096765
119713,688981,2021-03-31,2021,1,3,2.022254e+11,3.208473e+10,5.659924e+10,1.007814e+11,5.364424e+09,1.273850e+09,0.144340,1.469035
119714,688981,2021-06-30,2021,2,6,2.077294e+11,3.321193e+10,6.063528e+10,1.037010e+11,8.187185e+09,2.789781e+09,0.144340,1.469035
119715,688981,2021-09-30,2021,3,9,2.150651e+11,3.392805e+10,6.366095e+10,1.066986e+11,8.570357e+09,3.149870e+09,0.144340,1.469035


In [72]:
#剔除了未发布过年报的数据
len(style_factor.code.unique())

3978

In [73]:
#合并style_factor与disclosure,on = ['code','Accper']
style_factor1 = pd.merge(style_factor,disclosure,on = ['code','Accper'])

In [74]:
style_factor1.isnull().sum()

code              0
Accper            0
year              0
quarter           0
month             0
asset             0
debt_long         0
debt_all          0
equity            0
netincome_ttm4    0
OCF_ttm4          0
SGRO              3
EGRO              1
disclosure        0
dtype: int64

In [75]:
#找出SGRO空值，不予处理
style_factor1[style_factor1['SGRO'].isnull()]

,code,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4,SGRO,EGRO,disclosure
108656,601518,2010-12-31,2010,4,12,2.490867e+09,1.595000e+08,5.440683e+08,1.638718e+09,1.368719e+08,6.330772e+07,NaN,NaN,2011-03-31
119490,688266,2020-09-30,2020,3,9,2.002288e+09,4.262049e+06,2.159675e+08,1.705021e+09,-6.902495e+08,-7.039436e+08,NaN,-0.466522,2020-10-30
119491,688266,2020-12-31,2020,4,12,1.970282e+09,3.435228e+07,2.786922e+08,1.617725e+09,-3.192293e+08,-3.460472e+08,NaN,-0.466522,2021-04-29


In [76]:
style_factor1

,code,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4,SGRO,EGRO,disclosure
0,1,2010-12-31,2010,4,12,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,6.283816e+09,5.674904e+09,0.321171,2.082371,2011-02-25
1,1,2011-03-31,2011,1,3,8.075634e+11,9.474041e+09,7.716134e+11,3.595007e+10,7.107897e+09,6.421850e+09,0.354519,1.672216,2011-04-27
2,1,2011-06-30,2011,2,6,8.520570e+11,1.312510e+10,8.139046e+11,3.815240e+10,7.982443e+09,7.191803e+09,0.354519,1.672216,2011-08-18
3,1,2011-09-30,2011,3,9,1.207212e+12,1.604695e+10,1.135526e+12,6.970795e+10,9.237058e+09,8.623429e+09,0.354519,1.672216,2011-10-26
4,1,2011-12-31,2011,4,12,1.258177e+12,1.605449e+10,1.182796e+12,7.331084e+10,1.027863e+10,9.709968e+09,0.354519,1.672216,2012-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119644,688819,2021-12-31,2021,4,12,2.667708e+10,3.958674e+08,1.393798e+10,1.227199e+10,1.369366e+09,9.735234e+08,0.098626,0.096765,2022-03-31
119645,688981,2021-03-31,2021,1,3,2.022254e+11,3.208473e+10,5.659924e+10,1.007814e+11,5.364424e+09,1.273850e+09,0.144340,1.469035,2021-05-14
119646,688981,2021-06-30,2021,2,6,2.077294e+11,3.321193e+10,6.063528e+10,1.037010e+11,8.187185e+09,2.789781e+09,0.144340,1.469035,2021-08-28
119647,688981,2021-09-30,2021,3,9,2.150651e+11,3.392805e+10,6.366095e+10,1.066986e+11,8.570357e+09,3.149870e+09,0.144340,1.469035,2021-11-12


In [77]:
len(style_factor1.code.unique())

3978

In [78]:
#考虑某些公司某些年份一季报和财报公布时间相同，故作去重处理，保留最后一条也即最新的一条数据
style_factor1.drop_duplicates(subset=['code','disclosure'], keep='last',inplace=True)

In [79]:
len(style_factor1)

112118

In [80]:
#合并stktrd与style_factor1，left_on = ['code','date']， right_on = ['code','disclosure']
style_factor_all = pd.merge(stktrd, style_factor1, left_on = ['code','date'], right_on = ['code','disclosure'], how = 'left')

In [81]:
style_factor_all

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,CMRA,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4,SGRO,EGRO,disclosure
0,1,2010-01-04,23.710,24192276,69330749.18,73629834.50,-0.027082,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1,2010-01-05,23.300,55649982,68131862.33,72356606.65,-0.017292,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,1,2010-01-06,22.900,41214313,66962216.62,71114433.15,-0.017167,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,1,2010-01-07,22.650,35533685,66231188.06,70338074.71,-0.010917,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,1,2010-01-08,22.600,28854306,66084982.34,70182803.02,-0.002208,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.621,479100,114264.00,216729.00,-0.026646,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8672605,900957,2022-01-25,0.611,372779,112424.00,213239.00,-0.016103,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8672606,900957,2022-01-26,0.614,222600,112976.00,214286.00,0.004910,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8672607,900957,2022-01-27,0.612,245100,112608.00,213588.00,-0.003257,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [82]:
style_factor_all.isnull().sum()

code                    0
date                    0
close                   0
volume                  0
value_neg               0
value_all               0
return_day              0
shares_neg              0
CMRA              1002684
Accper            8588484
year              8588484
quarter           8588484
month             8588484
asset             8588484
debt_long         8588484
debt_all          8588484
equity            8588484
netincome_ttm4    8588484
OCF_ttm4          8588484
SGRO              8588487
EGRO              8588485
disclosure        8588484
dtype: int64

In [83]:
#分组，用行向下填充
style_factor_all[['Accper','year','quarter','month','asset','debt_long','debt_all','equity','netincome_ttm4','OCF_ttm4','SGRO','EGRO','disclosure']] = style_factor_all.groupby('code')[['Accper','year','quarter','month','asset','debt_long','debt_all','equity','netincome_ttm4','OCF_ttm4','SGRO','EGRO','disclosure']].fillna(method = 'ffill', axis=0)

In [84]:
style_factor_all

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,CMRA,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4,SGRO,EGRO,disclosure
0,1,2010-01-04,23.710,24192276,69330749.18,73629834.50,-0.027082,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1,2010-01-05,23.300,55649982,68131862.33,72356606.65,-0.017292,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,1,2010-01-06,22.900,41214313,66962216.62,71114433.15,-0.017167,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,1,2010-01-07,22.650,35533685,66231188.06,70338074.71,-0.010917,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,1,2010-01-08,22.600,28854306,66084982.34,70182803.02,-0.002208,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.621,479100,114264.00,216729.00,-0.026646,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8672605,900957,2022-01-25,0.611,372779,112424.00,213239.00,-0.016103,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8672606,900957,2022-01-26,0.614,222600,112976.00,214286.00,0.004910,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8672607,900957,2022-01-27,0.612,245100,112608.00,213588.00,-0.003257,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [85]:
#查看填充后依然有nan值得行
style_factor_all.isnull().sum()

code                    0
date                    0
close                   0
volume                  0
value_neg               0
value_all               0
return_day              0
shares_neg              0
CMRA              1002684
Accper            1458097
year              1458097
quarter           1458097
month             1458097
asset             1458097
debt_long         1458097
debt_all          1458097
equity            1458097
netincome_ttm4    1458097
OCF_ttm4          1458097
SGRO              1458502
EGRO              1458195
disclosure        1458097
dtype: int64

In [86]:
#若剔除空值行，还有3937只股票，由于交易数据到2022年1月，故在此之前财报数量不足4的企业netincome_ttm4和OCF_ttm4都应为nan
style_factor_all_dropna = style_factor_all.dropna()
len(style_factor_all_dropna.code.unique())

3937

### **四、计算风格因子**

style_factor_all数据集目前还有较多无财务数据的行，为了后续股价时间序列的计算，先不予删除

#### **1、Beta-BETA，Volatility-HSIGMA合并**

In [87]:
#读取计算好的beta和resid_std数据
BETA = pd.read_hdf('C:/Users/Administrator/Downloads/alpha191/ols_result.h5',key = 'ols_result')

In [88]:
#剔除不需要的列
BETA.drop(columns = ['return','Retindex','return_half','Retindex_half'], inplace = True)

In [89]:
#重命名列
BETA.rename(columns = {'Beta':'BETA', 'resid_std':'HSIGMA'}, inplace = True)

In [90]:
#合并style_factor_all与BETA
style_factor_all = style_factor_all.merge(BETA, on = ['code','date'])

In [91]:
style_factor_all.isnull().sum()

code                    0
date                    0
close                   0
volume                  0
value_neg               0
value_all               0
return_day              0
shares_neg              0
CMRA              1002684
Accper            1458097
year              1458097
quarter           1458097
month             1458097
asset             1458097
debt_long         1458097
debt_all          1458097
equity            1458097
netincome_ttm4    1458097
OCF_ttm4          1458097
SGRO              1458502
EGRO              1458195
disclosure        1458097
BETA              1141034
HSIGMA            1141034
dtype: int64

In [92]:
style_factor_all

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,CMRA,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4,SGRO,EGRO,disclosure,BETA,HSIGMA
0,1,2010-01-04,23.710,24192276,69330749.18,73629834.50,-0.027082,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,1,2010-01-05,23.300,55649982,68131862.33,72356606.65,-0.017292,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,1,2010-01-06,22.900,41214313,66962216.62,71114433.15,-0.017167,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,1,2010-01-07,22.650,35533685,66231188.06,70338074.71,-0.010917,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,1,2010-01-08,22.600,28854306,66084982.34,70182803.02,-0.002208,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.621,479100,114264.00,216729.00,-0.026646,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.668251,0.000730
8672605,900957,2022-01-25,0.611,372779,112424.00,213239.00,-0.016103,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.697025,0.000720
8672606,900957,2022-01-26,0.614,222600,112976.00,214286.00,0.004910,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.713335,0.000714
8672607,900957,2022-01-27,0.612,245100,112608.00,213588.00,-0.003257,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.726680,0.000709


#### **2、Volatility-DASTD合并**

In [93]:
#读取DASTD数据
DASTD = pd.read_hdf('C:/Users/Administrator/Downloads/alpha191/DASTD.h5', key = 'DASTD')

In [94]:
DASTD.drop(columns=['return_day','return_day_mean250','return_d_Sq_dev'],inplace=True)

In [95]:
DASTD.loc[:,'DASTD'] = DASTD['DASTD']**0.5

In [96]:
style_factor_all = pd.merge(style_factor_all,DASTD, on = ['code','date'], how = 'left')

In [97]:
style_factor_all.isnull().sum()#DASTD计算需要至少500个交易日数据，因此nan值较多

code                    0
date                    0
close                   0
volume                  0
value_neg               0
value_all               0
return_day              0
shares_neg              0
CMRA              1002684
Accper            1458097
year              1458097
quarter           1458097
month             1458097
asset             1458097
debt_long         1458097
debt_all          1458097
equity            1458097
netincome_ttm4    1458097
OCF_ttm4          1458097
SGRO              1458502
EGRO              1458195
disclosure        1458097
BETA              1141034
HSIGMA            1141034
DASTD             1654807
dtype: int64

In [98]:
style_factor_all

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,CMRA,Accper,year,quarter,month,asset,debt_long,debt_all,equity,netincome_ttm4,OCF_ttm4,SGRO,EGRO,disclosure,BETA,HSIGMA,DASTD
0,1,2010-01-04,23.710,24192276,69330749.18,73629834.50,-0.027082,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,1,2010-01-05,23.300,55649982,68131862.33,72356606.65,-0.017292,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,1,2010-01-06,22.900,41214313,66962216.62,71114433.15,-0.017167,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,1,2010-01-07,22.650,35533685,66231188.06,70338074.71,-0.010917,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,1,2010-01-08,22.600,28854306,66084982.34,70182803.02,-0.002208,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.621,479100,114264.00,216729.00,-0.026646,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.668251,0.000730,NaN
8672605,900957,2022-01-25,0.611,372779,112424.00,213239.00,-0.016103,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.697025,0.000720,NaN
8672606,900957,2022-01-26,0.614,222600,112976.00,214286.00,0.004910,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.713335,0.000714,NaN
8672607,900957,2022-01-27,0.612,245100,112608.00,213588.00,-0.003257,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.726680,0.000709,NaN


#### **2、Momentum-RSTR合并**

In [99]:
#读取计算好的RSTR数据
RSTR = pd.read_hdf('C:/Users/Administrator/Downloads/alpha191/RSTR.h5',key = 'RSTR')

In [100]:
#剔除不需要的列
RSTR.drop(columns = ['return_day','ln_return_day','RSTR_0'], inplace = True)

In [101]:
RSTR

,code,date,RSTR
0,1,2008-01-02,NaN
1,1,2008-01-03,NaN
2,1,2008-01-04,NaN
3,1,2008-01-07,NaN
4,1,2008-01-08,NaN
...,...,...,...
9127532,689009,2022-01-24,NaN
9127533,689009,2022-01-25,NaN
9127534,689009,2022-01-26,NaN
9127535,689009,2022-01-27,NaN


In [102]:
#由于计算RSTR的windows为500，故上市交易日期不足500的股票未来都将被删除
style_factor_all = style_factor_all.merge(RSTR, on = ['code','date'], how = 'left')

In [103]:
style_factor_all.isnull().sum()#RSTR计算需要至少520个交易日数据，因此nan值较多

code                    0
date                    0
close                   0
volume                  0
value_neg               0
value_all               0
return_day              0
shares_neg              0
CMRA              1002684
Accper            1458097
year              1458097
quarter           1458097
month             1458097
asset             1458097
debt_long         1458097
debt_all          1458097
equity            1458097
netincome_ttm4    1458097
OCF_ttm4          1458097
SGRO              1458502
EGRO              1458195
disclosure        1458097
BETA              1141034
HSIGMA            1141034
DASTD             1654807
RSTR              1738328
dtype: int64

#### **3、Size-LNCAP计算**

In [104]:
style_factor_all.loc[:,'LNCAP'] = np.log(style_factor_all['value_all'])

#### **4、EarningsYield-ETOP、CETOP计算**

In [105]:
style_factor_all.loc[:,'ETOP'] = style_factor_all['netincome_ttm4']/style_factor_all['value_all']

In [106]:
style_factor_all.loc[:,'CETOP'] = style_factor_all['OCF_ttm4']/style_factor_all['value_all']

In [107]:
#删除不需要的列
style_factor_all.drop(columns = ['netincome_ttm4','OCF_ttm4'], inplace = True)

#### **5、Leverage-MLEV、DTOA、BLEV计算**

In [108]:
style_factor_all.loc[:,'MLEV'] = (style_factor_all['value_all'] + style_factor_all['debt_long'])/style_factor_all['value_all']

In [109]:
style_factor_all.loc[:,'DTOA'] = style_factor_all['debt_all']/style_factor_all['asset']

In [110]:
style_factor_all.loc[:,'BLEV'] = (style_factor_all['equity'] + style_factor_all['debt_long'])/style_factor_all['equity']

#### **6、Value-BTOP计算**

In [111]:
style_factor_all.loc[:,'BTOP'] = style_factor_all['equity']/style_factor_all['value_all']

#### **7、Liquidity-STOM、STOQ、STOA计算**

In [112]:
#计算月换手率，先计算当日成交量volume除以流通股本数shares_neg，记为volume/shares_neg
style_factor_all.loc[:,'volume/shares_neg'] = style_factor_all['volume']/style_factor_all['shares_neg']

In [113]:
#计算volume/shares_neg的21天之和，记为vo/s_sum21
style_factor_all.loc[:,'vo/s_sum21'] = style_factor_all.groupby('code',as_index=False)['volume/shares_neg'].rolling(21).sum()['volume/shares_neg']

In [114]:
#计算STOM（月换手率）
style_factor_all.loc[:,'STOM'] = np.log(style_factor_all['vo/s_sum21'])

In [115]:
#计算volume/shares_neg的63天之和，记为vo/s_sum21
style_factor_all.loc[:,'vo/s_sum63'] = style_factor_all.groupby('code',as_index=False)['volume/shares_neg'].rolling(63).sum()['volume/shares_neg']

In [116]:
#计算STOQ（季度平均月换手率）
style_factor_all.loc[:,'STOQ'] = np.log(style_factor_all['vo/s_sum63']/3)

In [117]:
#计算volume/shares_neg的252天之和，记为vo/s_sum252
style_factor_all.loc[:,'vo/s_sum252'] = style_factor_all.groupby('code',as_index=False)['volume/shares_neg'].rolling(252).sum()['volume/shares_neg']

In [118]:
#计算STOA（年度平均月换手率）
style_factor_all.loc[:,'STOA'] = np.log(style_factor_all['vo/s_sum252']/12)

In [119]:
#删除不需要的列
style_factor_all.drop(columns = ['volume/shares_neg','vo/s_sum21','vo/s_sum63','vo/s_sum252'], inplace = True)

#### **9、小类因子合成大类因子**

In [120]:
#EarningsYield合成，由于EPIBS未计算，将ETOP和CETOP按照1：2合成
style_factor_all.loc[:,'EarningsYield'] = 0.34*style_factor_all['ETOP'] + 0.66*style_factor_all['CETOP']

In [121]:
#Volatility合成，Volatility=0.74DASTD+0.16CMRA+0.10HSIGMA
style_factor_all.loc[:,'Volatility'] = 0.74*style_factor_all['DASTD']+0.16*style_factor_all['CMRA']+0.10*style_factor_all['HSIGMA']#指标合成需要DASTD和CMRA，因此空值较多

In [122]:
#Growth合成，由于EGIB、EGIB_S未计算，将EGRO和SGRO按照1：2合成
style_factor_all.loc[:,'Growth'] = 0.34*style_factor_all['EGRO']+0.66*style_factor_all['SGRO']

In [123]:
#Leverage合成，Leverage=0.38*MLEV+0.35*DTOA+0.27*BLEV
style_factor_all.loc[:,'Leverage'] = 0.38*style_factor_all['MLEV']+0.35*style_factor_all['DTOA']+0.27*style_factor_all['BLEV']

In [124]:
#Liquidity合成，Liquidity=0.35*STOM+0.35*STOQ+0.3*STOA
style_factor_all.loc[:,'Liquidity'] = 0.35*style_factor_all['STOM']+0.35*style_factor_all['STOQ']+0.3*style_factor_all['STOA']

#### **10、空值查询**

In [125]:
#若剔除RSTR空值行，还有3783只股票，计算RSTR的windows为500，由于交易数据到2022年1月，故在此之前上市交易日期不足500的股票未来都将被删除
style_factor_all_dropna = style_factor_all.drop(style_factor_all[style_factor_all['RSTR'].isnull()].index)
len(style_factor_all_dropna.code.unique())

3783

In [126]:
#若剔除空值行，还有3777只股票
style_factor_all_dropna = style_factor_all.dropna()
print(len(style_factor_all_dropna.code.unique()))
print(len(style_factor_all_dropna))

3777
6535958


In [127]:
style_factor_all.isnull().sum()

code                   0
date                   0
close                  0
volume                 0
value_neg              0
value_all              0
return_day             0
shares_neg             0
CMRA             1002684
Accper           1458097
year             1458097
quarter          1458097
month            1458097
asset            1458097
debt_long        1458097
debt_all         1458097
equity           1458097
SGRO             1458502
EGRO             1458195
disclosure       1458097
BETA             1141034
HSIGMA           1141034
DASTD            1654807
RSTR             1738328
LNCAP                  0
ETOP             1458097
CETOP            1458097
MLEV             1458097
DTOA             1458097
BLEV             1458209
BTOP             1458097
STOM               96118
STOQ              295584
STOA             1149677
EarningsYield    1458097
Volatility       1993029
Growth           1458502
Leverage         1458209
Liquidity        1149677
dtype: int64

In [128]:
style_factor_all

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,CMRA,Accper,year,quarter,month,asset,debt_long,debt_all,equity,SGRO,EGRO,disclosure,BETA,HSIGMA,DASTD,RSTR,LNCAP,ETOP,CETOP,MLEV,DTOA,BLEV,BTOP,STOM,STOQ,STOA,EarningsYield,Volatility,Growth,Leverage,Liquidity
0,1,2010-01-04,23.710,24192276,69330749.18,73629834.50,-0.027082,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,18.114561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2010-01-05,23.300,55649982,68131862.33,72356606.65,-0.017292,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,18.097117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2010-01-06,22.900,41214313,66962216.62,71114433.15,-0.017167,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,18.079801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2010-01-07,22.650,35533685,66231188.06,70338074.71,-0.010917,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,18.068824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2010-01-08,22.600,28854306,66084982.34,70182803.02,-0.002208,2.924114e+06,1.603651,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,18.066614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.621,479100,114264.00,216729.00,-0.026646,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.668251,0.000730,NaN,NaN,12.286403,NaN,NaN,NaN,NaN,NaN,NaN,3.430993,3.092180,3.895910,NaN,NaN,NaN,NaN,3.451884
8672605,900957,2022-01-25,0.611,372779,112424.00,213239.00,-0.016103,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.697025,0.000720,NaN,NaN,12.270169,NaN,NaN,NaN,NaN,NaN,NaN,3.474904,3.109855,3.894905,NaN,NaN,NaN,NaN,3.473137
8672606,900957,2022-01-26,0.614,222600,112976.00,214286.00,0.004910,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.713335,0.000714,NaN,NaN,12.275067,NaN,NaN,NaN,NaN,NaN,NaN,3.429122,3.102580,3.892927,NaN,NaN,NaN,NaN,3.453974
8672607,900957,2022-01-27,0.612,245100,112608.00,213588.00,-0.003257,1.840000e+05,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1.726680,0.000709,NaN,NaN,12.271804,NaN,NaN,NaN,NaN,NaN,NaN,3.440926,3.114568,3.888057,NaN,NaN,NaN,NaN,3.460840


### **五、因子预处理：去空值**

#### **1、去除空值并输出codelist作为筛选标准，同时确保时间序列的连续性,因此删除每只股前520行，再检查后删除剩余4310个空值行**

In [129]:
#选出需要的大类指标
style_factor_dropna = style_factor_all.dropna()

In [130]:
style_factor_dropna

,code,date,close,volume,value_neg,value_all,return_day,shares_neg,CMRA,Accper,year,quarter,month,asset,debt_long,debt_all,equity,SGRO,EGRO,disclosure,BETA,HSIGMA,DASTD,RSTR,LNCAP,ETOP,CETOP,MLEV,DTOA,BLEV,BTOP,STOM,STOQ,STOA,EarningsYield,Volatility,Growth,Leverage,Liquidity
251,1,2011-04-06,17.00,74524386,52791094.69,59245233.95,0.025951,3.105359e+06,0.854070,2010-12-31,2010.0,4.0,12.0,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,0.321171,2.082371,2011-02-25,2.219435,0.001162,0.017506,-0.000308,17.897196,106.064498,95.786676,160.835439,0.954349,1.285247,560.341023,5.319873,5.290394,5.524013,99.281135,0.149722,0.919979,61.798506,5.370797
252,1,2011-04-07,17.20,64881087,53412166.39,59942236.71,0.011765,3.105359e+06,0.854070,2010-12-31,2010.0,4.0,12.0,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,0.321171,2.082371,2011-02-25,2.215657,0.001152,0.017432,-0.000280,17.908892,104.831190,94.672877,158.976888,0.954349,1.285247,553.825430,5.343417,5.312437,5.528200,98.126703,0.149666,0.919979,61.092256,5.388009
253,1,2011-04-08,17.17,26811761,53319005.63,59837686.29,-0.001744,3.105359e+06,0.854070,2010-12-31,2010.0,4.0,12.0,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,0.321171,2.082371,2011-02-25,2.212386,0.001145,0.017281,-0.000320,17.907146,105.014354,94.838293,159.252910,0.954349,1.285247,554.793092,5.335119,5.317180,5.524751,98.298154,0.149554,0.919979,61.197145,5.385730
254,1,2011-04-11,17.03,56381824,52884255.44,59349784.37,-0.008154,3.105359e+06,0.854070,2010-12-31,2010.0,4.0,12.0,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,0.321171,2.082371,2011-02-25,2.208936,0.001137,0.017158,-0.000321,17.898959,105.877655,95.617938,160.553874,0.954349,1.285247,559.353928,5.371714,5.334045,5.526100,99.106242,0.149462,0.919979,61.691511,5.404846
255,1,2011-04-12,17.06,38618173,52977416.20,59454334.78,0.001762,3.105359e+06,0.854070,2010-12-31,2010.0,4.0,12.0,7.272071e+11,9.469488e+09,6.940095e+11,3.319754e+10,0.321171,2.082371,2011-02-25,2.205396,0.001127,0.017014,-0.000493,17.900719,105.691469,95.449794,160.273298,0.954349,1.285247,558.370304,5.383270,5.335460,5.526194,98.931964,0.149354,0.919979,61.584892,5.409414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8490871,688388,2022-01-28,120.64,2637989,20613465.21,28252920.83,-0.031782,1.708676e+05,0.984275,2021-09-30,2021.0,3.0,9.0,5.332814e+09,1.026814e+09,2.134285e+09,3.185892e+09,0.561339,0.765428,2021-10-20,9.016210,0.003163,0.040696,0.001819,17.156707,16.427853,15.306719,37.343648,0.400217,1.322300,112.763262,6.232095,6.393542,6.335343,15.687904,0.187915,0.630729,14.687683,6.319576
8493770,688399,2022-01-25,135.01,1469199,4654442.75,7914286.20,-0.036538,3.447480e+04,1.572991,2020-12-31,2020.0,4.0,12.0,2.277673e+09,0.000000e+00,6.131765e+08,1.664497e+09,1.489448,3.424160,2021-03-26,3.952740,0.001567,0.051261,-0.000538,15.884180,103.555434,97.040521,1.000000,0.269212,1.000000,210.315468,7.586344,6.996476,6.643402,99.255592,0.289768,2.147250,0.744224,7.097008
8493771,688399,2022-01-26,133.20,1071227,4592043.36,7808184.00,-0.013406,3.447480e+04,1.572991,2020-12-31,2020.0,4.0,12.0,2.277673e+09,0.000000e+00,6.131765e+08,1.664497e+09,1.489448,3.424160,2021-03-26,3.938729,0.001571,0.050847,-0.000325,15.870683,104.962606,98.359165,1.000000,0.269212,1.000000,213.173359,7.577892,6.983523,6.643177,100.604335,0.289463,2.147250,0.744224,7.089448
8493772,688399,2022-01-27,130.61,1461386,4502753.63,7656358.20,-0.019444,3.447480e+04,1.572991,2020-12-31,2020.0,4.0,12.0,2.277673e+09,0.000000e+00,6.131765e+08,1.664497e+09,1.489448,3.424160,2021-03-26,3.926429,0.001576,0.050472,-0.000050,15.851047,107.044017,100.309630,1.000000,0.269212,1.000000,217.400592,7.569063,6.981237,6.644440,102.599322,0.289186,2.147250,0.744224,7.085937


In [131]:
#重置索引
style_factor_dropna.reset_index(drop=True,inplace=True)

In [132]:
#若剔除空值行，还有3777只股票
print(len(style_factor_dropna.code.unique()))

3777


In [133]:
#导出code列表
codelist = pd.Series(style_factor_dropna.code.unique())

In [134]:
codelist

0            1
1            2
2            4
3            5
4            6
         ...  
3772    688366
3773    688368
3774    688369
3775    688388
3776    688399
Length: 3777, dtype: int64

In [135]:
#筛选出codelist里的股票，并选出需要的列（大类因子）
style_factor_final = style_factor_all[style_factor_all['code'].isin(codelist)][['code','date','BETA','RSTR','LNCAP','EarningsYield','Volatility','Growth','BTOP','Leverage','Liquidity']]

In [136]:
#大类因子重命名
style_factor_final.rename(columns={'BETA':'Beta','RSTR':'Momentum','LNCAP':'Size','BTOP':'Value'}, inplace=True)

In [137]:
style_factor_final.isnull().sum()

code                   0
date                   0
Beta              940473
Momentum         1233103
Size                   0
EarningsYield     986170
Volatility       1487812
Growth            986268
Value             986170
Leverage          986282
Liquidity         948027
dtype: int64

In [159]:
#删除每只股票前520行
style_factor_f520 = style_factor_final.groupby('code',as_index = False).apply(lambda x: x.drop(x.head(520).index))

In [236]:
#重置索引
style_factor_f520.reset_index(drop = True, inplace=True)

In [237]:
style_factor_f520

,code,date,Beta,Momentum,Size,EarningsYield,Volatility,Growth,Value,Leverage,Liquidity
0,1,2012-06-08,0.553191,-0.000330,18.144622,143.800440,0.042350,0.729137,1012.368380,81.473270,4.859730
1,1,2012-06-11,0.554001,-0.000330,18.143270,143.994896,0.042265,0.729137,1013.737370,81.582042,4.870956
2,1,2012-06-12,0.553496,-0.000364,18.125535,146.571543,0.042317,0.729137,1031.877200,83.023330,4.885374
3,1,2012-06-13,0.553732,-0.000386,18.146645,143.509738,0.042460,0.729137,1010.321813,81.310662,4.889370
4,1,2012-06-14,0.552202,-0.000350,18.133077,145.470254,0.042447,0.729137,1024.124024,82.407307,4.877098
...,...,...,...,...,...,...,...,...,...,...,...
6203152,688388,2022-01-28,9.016210,0.001819,17.156707,15.687904,0.187915,0.630729,112.763262,14.687683,6.319576
6203153,688399,2022-01-25,3.952740,-0.000538,15.884180,99.255592,0.289768,2.147250,210.315468,0.744224,7.097008
6203154,688399,2022-01-26,3.938729,-0.000325,15.870683,100.604335,0.289463,2.147250,213.173359,0.744224,7.089448
6203155,688399,2022-01-27,3.926429,-0.000050,15.851047,102.599322,0.289186,2.147250,217.400592,0.744224,7.085937


In [238]:
style_factor_f520.isnull().sum()
#在剔除时间序列前520行数据以后，出现EarningsYield、Growth、Value、
#Leverage缺失值都是4310的情况，且这部分指标都与财报数据有关，说明：
#虽然某些股票已经有了较长时间的交易数据，但其财务报表数据仍然缺失，缺失原因不详。

code                0
date                0
Beta                0
Momentum            0
Size                0
EarningsYield    4310
Volatility          0
Growth           4310
Value            4310
Leverage         4310
Liquidity           0
dtype: int64

In [239]:
#查询空值行
style_factor_f520[style_factor_f520['EarningsYield'].isnull()].code.unique()
#经验证，该些股票的财务数据均缺失，且与前520个缺失行连续，因此可直接删除，不会破坏时间序列连续性,可直接dropna

array([     4,    538,    573,    611,    626,    690,    711,    821,
          833,    971,   2005,   2194,   2200, 600030, 600037, 600126,
       600130, 600209, 600470, 600640, 600645, 600680, 600693, 600715,
       600760, 600856, 600877, 600891, 601088, 601229, 601688, 601888,
       603568], dtype=int64)

In [241]:
style_factor_f520.dropna(inplace=True)

In [242]:
style_factor_f520.reset_index(drop = True, inplace= True)

In [246]:
style_factor_f520.isnull().sum()

code             0
date             0
Beta             0
Momentum         0
Size             0
EarningsYield    0
Volatility       0
Growth           0
Value            0
Leverage         0
Liquidity        0
dtype: int64

In [262]:
style_factor_f520

,code,date,Beta,Momentum,Size,EarningsYield,Volatility,Growth,Value,Leverage,Liquidity
0,1,2012-06-08,0.553191,-0.000330,18.144622,143.800440,0.042350,0.729137,1012.368380,81.473270,4.859730
1,1,2012-06-11,0.554001,-0.000330,18.143270,143.994896,0.042265,0.729137,1013.737370,81.582042,4.870956
2,1,2012-06-12,0.553496,-0.000364,18.125535,146.571543,0.042317,0.729137,1031.877200,83.023330,4.885374
3,1,2012-06-13,0.553732,-0.000386,18.146645,143.509738,0.042460,0.729137,1010.321813,81.310662,4.889370
4,1,2012-06-14,0.552202,-0.000350,18.133077,145.470254,0.042447,0.729137,1024.124024,82.407307,4.877098
...,...,...,...,...,...,...,...,...,...,...,...
6198842,688388,2022-01-28,9.016210,0.001819,17.156707,15.687904,0.187915,0.630729,112.763262,14.687683,6.319576
6198843,688399,2022-01-25,3.952740,-0.000538,15.884180,99.255592,0.289768,2.147250,210.315468,0.744224,7.097008
6198844,688399,2022-01-26,3.938729,-0.000325,15.870683,100.604335,0.289463,2.147250,213.173359,0.744224,7.089448
6198845,688399,2022-01-27,3.926429,-0.000050,15.851047,102.599322,0.289186,2.147250,217.400592,0.744224,7.085937


In [248]:
style_factor_f520.to_hdf('C:/Users/Administrator/Downloads/alpha191/style_factor_f520.h5', key = 'style_factor_f520')

#### **2、先不对style_factor进行去极值、标准化处理**

In [250]:
#函数：对因子数据进行MAD去极值处理（.value_counts()）
def filter_extreme_MAD(series,n=5):
    md = np.median(series)
    mad = np.median((series-md).abs())
    max_range = md + n*mad
    min_range = md - n*mad
    return np.clip(series,min_range,max_range)

In [3]:
style_factor_f520 = pd.read_hdf('C:/Users/Administrator/Downloads/alpha191/style_factor_f520.h5', key = 'style_factor_f520')

In [5]:
style_factor_f520.describe()

,code,Beta,Momentum,Size,EarningsYield,Volatility,Growth,Value,Leverage,Liquidity
count,6.198847e+06,6.198847e+06,6.198847e+06,6.198847e+06,6.198847e+06,6.198847e+06,6.198847e+06,6.198847e+06,6.198847e+06,6.198847e+06
mean,2.919287e+05,7.438738e-01,1.358213e-04,1.575054e+01,1.279087e+01,1.380631e-01,3.580192e+00,4.603580e+02,6.451992e+01,5.801628e+00
std,2.739945e+05,1.523708e+00,1.590817e-03,1.053185e+00,1.625884e+02,6.086620e-02,2.539299e+02,6.790455e+02,1.923324e+02,8.498730e-01
min,1.000000e+00,-8.184751e+01,-1.668044e-02,1.113711e+01,-2.395906e+04,1.618705e-02,-7.722896e+03,-1.432276e+04,-6.098843e+01,3.380475e-01
25%,2.234000e+03,9.738896e-02,-8.398726e-04,1.501213e+01,2.045004e+00,9.465283e-02,1.240247e-02,2.183947e+02,8.086170e-01,5.251314e+00
50%,3.001960e+05,7.873382e-01,-4.780293e-05,1.558110e+01,1.661018e+01,1.260009e-01,1.575688e-01,3.675288e+02,7.467726e+00,5.830535e+00
75%,6.005120e+05,1.375388e+00,9.451249e-04,1.631692e+01,3.582212e+01,1.692052e-01,3.327903e-01,5.868250e+02,4.718859e+01,6.402281e+00
max,6.883990e+05,7.219236e+01,1.643795e-02,2.190725e+01,7.089869e+03,2.396564e+00,1.776843e+04,4.486484e+04,6.878939e+03,8.784367e+00
